In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.linalg import svd

from grakel.kernels import WeisfeilerLehman
from grakel.utils import graph_from_networkx
from netrd.distance import Frobenius
from netrd.distance import JaccardDistance
from netrd.distance import DeltaCon

## Create Directed Trade Networks of Aggregated Import Value

In [2]:
temp = pd.read_stata("../data/country_partner_sitcproductsection_year.dta")

In [64]:
temp

,location_id,partner_id,product_id,year,export_value,import_value,sitc_eci,sitc_coi,location_code,partner_code,sitc_product_code,Total_Trade,Trade_Balance
0,0,1,0,2018.0,46139.0,0.0,0.800212,0.174964,ABW,AFG,0,46139.0,46139.0
1,0,1,0,2019.0,62796.0,0.0,0.935727,-0.282090,ABW,AFG,0,62796.0,62796.0
2,2,1,0,2007.0,0.0,5560.0,-1.794320,-1.203246,AGO,AFG,0,5560.0,-5560.0
3,2,1,0,2014.0,0.0,41341.0,-1.547068,-1.121438,AGO,AFG,0,41341.0,-41341.0
4,2,1,0,2009.0,0.0,29262.0,-1.607222,-1.188794,AGO,AFG,0,29262.0,-29262.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7290683,192,235,4,1971.0,280361.0,350105.0,0.540057,0.916770,SGP,VDR,4,630466.0,-69744.0
7290684,192,235,4,1972.0,144771.0,298686.0,0.461371,0.925253,SGP,VDR,4,443457.0,-153915.0
7290685,192,235,4,1973.0,771424.0,537051.0,0.656068,1.318713,SGP,VDR,4,1308475.0,234373.0
7290686,192,235,4,1974.0,1721284.0,638357.0,0.689488,1.429749,SGP,VDR,4,2359641.0,1082927.0


In [65]:
# Group by 'year', 'country1', 'country2' and sum 'export_value' and 'import_value'
df_sum = temp.groupby(['year', 'location_id', 'partner_id']).agg({
    'export_value': 'sum',
    'import_value': 'sum'
}).reset_index()

# Group by 'country1', 'country2' and take the mean of 'export_value' and 'import_value'
df_avg = df_sum.groupby(['location_id', 'partner_id']).agg({
    'export_value': 'mean',
    'import_value': 'mean'
}).reset_index()

print(df_avg)

       location_id  partner_id  export_value  import_value
0                0           1  6.671560e+05  0.000000e+00
1                0           2  1.101219e+07  0.000000e+00
2                0           4  1.417900e+04  0.000000e+00
3                0           6  1.622047e+07  2.283780e+07
4                0           7  2.833217e+05  2.681281e+06
...            ...         ...           ...           ...
47391          250         245  5.558290e+06  4.532469e+05
47392          250         246  9.511099e+08  4.884570e+09
47393          250         247  4.331187e+07  1.551284e+07
47394          250         248  1.701076e+08  1.970675e+08
47395          250         249  3.753082e+09  1.167818e+09

[47396 rows x 4 columns]


In [66]:
# Create a new column 'sorted_pair' that contains the sorted pair ('location_id', 'partner_id')
df_avg['sorted_pair'] = df_avg.apply(lambda row: tuple(sorted([row['location_id'], row['partner_id']])), axis=1)

# Drop duplicates based on 'sorted_pair'
df_avg = df_avg.drop_duplicates(subset=['sorted_pair'])

# Drop the 'sorted_pair' column as it's no longer needed
df_avg = df_avg.drop(columns=['sorted_pair'])

print(df_avg)

       location_id  partner_id  export_value  import_value
0                0           1  6.671560e+05  0.000000e+00
1                0           2  1.101219e+07  0.000000e+00
2                0           4  1.417900e+04  0.000000e+00
3                0           6  1.622047e+07  2.283780e+07
4                0           7  2.833217e+05  2.681281e+06
...            ...         ...           ...           ...
46715          247         249  3.796798e+07  2.818680e+06
46716          247         250  1.551284e+07  4.331187e+07
46939          248         249  1.004558e+07  6.699753e+06
46940          248         250  1.970675e+08  1.701076e+08
47182          249         250  1.167818e+09  3.753082e+09

[23698 rows x 4 columns]


In [78]:
df_avg['import_value'].describe()['75%']/10000

757.5020825

In [79]:
df_avg['Total_Trade'] = df_avg['import_value'] + df_avg['export_value']

In [85]:
df_avg['Trade_Balance'] = df_avg['export_value'] - df_avg['import_value']

In [90]:
df_avg['Trade_Balance'].describe()['75%']/10000

73.02873503016592

In [3]:
country_codes = pd.read_excel("../data/ISO3166.xlsx")

#rename some columns for join with trade data
country_codes["location_code"] = country_codes["Alpha-3 code"]
country_codes["partner_code"] = country_codes["Alpha-3 code"]
country_codes["country_i"] = country_codes["English short name"]
country_codes["country_j"] = country_codes["English short name"]

In [4]:
def clean_imports(df):
    #join trade data and descriptions
    df = pd.merge(df, country_codes[["location_code", "country_i"]],on = ["location_code"])
    df = pd.merge(df, country_codes[["partner_code", "country_j"]],on = ["partner_code"])
    #df = pd.merge(df, product_codes[["sitc_product_code", "product"]], on = ["sitc_product_code"])

    imports1 = df[['location_id', 'partner_id', 'product_id', 'year',
       'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
       'sitc_product_code', 'country_i', 'country_j']]
    imports1 = imports1[imports1["import_value"] != 0]

    imports2 = df[['location_id', 'partner_id', 'product_id', 'year',
       'export_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
       'sitc_product_code', 'country_i', 'country_j']]
    imports2["temp1"] = imports2['partner_code']
    imports2["temp2"] = imports2['location_code']

    imports2['location_code'] = imports2["temp1"]
    imports2['partner_code'] = imports2["temp2"]
    imports2["import_value"] = imports2["export_value"]
    imports2 = imports2[imports2["import_value"] != 0]
    imports2 = imports1[['location_id', 'partner_id', 'product_id', 'year',
        'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
        'sitc_product_code', 'country_i', 'country_j']]
    
    imports = pd.concat([imports1, imports2], ignore_index=True)
    
    return imports

In [5]:
def clean_exports(df):
    #join trade data and descriptions
    df = pd.merge(df, country_codes[["location_code", "country_i"]],on = ["location_code"])
    df = pd.merge(df, country_codes[["partner_code", "country_j"]],on = ["partner_code"])
    #df = pd.merge(df, product_codes[["sitc_product_code", "product"]], on = ["sitc_product_code"])

    exports1 = df[['location_id', 'partner_id', 'product_id', 'year',
       'export_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
       'sitc_product_code', 'country_i', 'country_j']]
    exports1 = exports1[exports1["export_value"] != 0]

    exports2 = df[['location_id', 'partner_id', 'product_id', 'year',
       'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
       'sitc_product_code', 'country_i', 'country_j']]
    exports2["temp1"] = exports2['partner_code']
    exports2["temp2"] = exports2['location_code']

    exports2['location_code'] = exports2["temp1"]
    exports2['partner_code'] = exports2["temp2"]
    exports2["export_value"] = exports2["import_value"]
    exports2 = exports2[exports2["export_value"] != 0]
    exports2 = exports1[['location_id', 'partner_id', 'product_id', 'year',
        'export_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
        'sitc_product_code', 'country_i', 'country_j']]
    
    exports = pd.concat([exports1, exports2], ignore_index=True)
    
    return exports

In [6]:
locations = pd.read_stata("../location_classifications/location.dta")
locations['location_id'] = locations['location_id'].astype(int)
locations.drop(columns = ['location_id','location_name_short_en','level'], inplace = True)

In [ ]:
#import trade data
export_data = []
for i in range(1962,2022):
    file_name = (f"../data/country_partner_sitcproduct4digit_year_{i}.dta")
    year_data = pd.read_stata(file_name)
    trade = clean_exports(year_data)
    
    trade = trade.merge(locations, left_on = "partner_code", right_on = "location_code")
    trade.rename(columns = {"location_code_x":"location_code"}, inplace = True)
    trade.drop(columns = ["location_code_y"], inplace = True)

    export_data.append(trade)

In [ ]:
#import trade data
import_data = []
for i in range(1962,2022):
    file_name = (f"data/country_partner_sitcproduct4digit_year_{i}.dta")
    year_data = pd.read_stata(file_name)
    trade = clean_imports(year_data)
    
    trade = trade.merge(locations, left_on = "partner_code", right_on = "location_code")
    trade.rename(columns = {"location_code_x":"location_code"}, inplace = True)
    trade.drop(columns = ["location_code_y"], inplace = True)

    import_data.append(trade)

In [ ]:
#Group dataframe by country
import_agg = []
for i in import_data:
    import_agg.append(i.groupby(["location_code","partner_code"])['import_value'].sum().reset_index())

In [ ]:
#Group dataframe by country
export_agg = []
for i in export_data:
    export_agg.append(i.groupby(["location_code","partner_code"])['export_value'].sum().reset_index())

In [6]:
with open('../graphs/exports_agg.pkl', 'rb') as file:
    export_agg = pkl.load(file)

In [7]:
with open('../graphs/imports_agg.pkl', 'rb') as file:
    import_agg = pkl.load(file)

## Standardizing Node Length of Aggregated Dataframes

In [11]:
export_partners = []
for i in export_agg:
    df = i.pivot(index='location_code', columns='partner_code', values='export_value')
    df = df.fillna(0)
    export_partners.append(df)

In [12]:
import_partners = []
for i in import_agg:
    df = i.pivot(index='location_code', columns='partner_code', values='import_value')
    df = df.fillna(0)
    import_partners.append(df)

In [13]:
import_percent = []
for i in import_partners:
    df = i.div(i.sum(axis=1), axis=0)
    import_percent.append(df)

In [14]:
export_percent = []
for i in export_partners:
    df = i.div(i.sum(axis=1), axis=0)
    export_percent.append(df)

In [15]:
max_index = []
max_len = len(export_percent[0].index)
for i in export_percent:
    if len(i.index) > max_len:
        max_index = i.index

In [16]:
for i in export_percent:
    for j in max_index:
        if j not in i.columns:
            i[j] = 0
        if j not in i.index:
            i.loc[j] = 0

In [17]:
max_columns = export_percent[0].columns
max_len = len(export_percent[0].columns)
for i in export_percent:
    if len(i.columns) > max_len:
        max_columns = i.columns

In [18]:
for i in export_percent:
    for j in max_columns:
        if j not in i.columns:
            i[j] = 0
        if j not in i.index:
            i.loc[j] = 0

In [19]:
for i in export_percent:
    if len(i) != 240:
        missing_cols = export_percent[38].columns.difference(i.columns)
        for j in missing_cols:
            i[j] = 0
            i.loc[j] = 0

In [20]:
for i in export_percent:
    if len(i.index)!= 240:
        missing_index = export_percent[38].index.difference(i.index)
        for j in missing_index:
            i.loc[j] = 0

In [21]:
max_index = []
max_len = len(import_percent[0].index)
for i in import_percent:
    if len(i.index) > max_len:
        max_index = i.index

In [22]:
for i in import_percent:
    for j in max_index:
        if j not in i.columns:
            i[j] = 0
        if j not in i.index:
            i.loc[j] = 0

In [23]:
max_columns = import_percent[0].columns
max_len = len(import_percent[0].columns)
for i in import_percent:
    if len(i.columns) > max_len:
        max_columns = i.columns

In [24]:
for i in import_percent:
    for j in max_columns:
        if j not in i.columns:
            i[j] = 0
        if j not in i.index:
            i.loc[j] = 0

/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_98678/2263629442.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  i[j] = 0


In [25]:
for i in import_percent:
    if len(i) != 240:
        missing_cols = import_percent[38].columns.difference(i.columns)
        for j in missing_cols:
            i[j] = 0
            i.loc[j] = 0

/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_98678/944248445.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  i[j] = 0


In [26]:
for i in import_percent:
    if len(i.index) != 240:
        missing_index = import_percent[38].columns.difference(i.columns)
        for j in missing_cols:
            i[j] = 0
            i.loc[j] = 0

In [27]:
export_graphs = []

for y in export_percent:
    G = nx.from_pandas_adjacency(y, create_using=nx.DiGraph())
    export_graphs.append(G)

In [30]:
import_graphs = []

for y in import_percent:
    G = nx.from_pandas_adjacency(y, create_using=nx.DiGraph())
    import_graphs.append(G)

In [31]:
years = range(1962,2022)
for i, g in enumerate(export_graphs):
    nx.set_edge_attributes(g, f'<[{years[i]}, {years[i]+1}]>', 'timeset')

In [32]:
graph_df = pd.DataFrame()
for i, g in enumerate(export_graphs):
    graph_df = pd.concat([graph_df,nx.to_pandas_edgelist(g)], ignore_index=True)

In [33]:
graph_df = graph_df.rename(columns = {'source':'Source','target':'Target'})

In [34]:
df_sorted = graph_df.sort_values(by=['Source','timeset','weight'], ascending=[True, True,False])

# Group by country and get the top two rows for each group
top_two_rows_per_country = df_sorted.groupby(['Source','timeset']).head(2)

In [35]:
top_countries_2021 = top_two_rows_per_country[top_two_rows_per_country['timeset'] == '<[2021, 2022]>']

In [36]:
regions = locations['parent_id'].unique()
regions = regions[np.logical_and(~np.isnan(regions), regions != 358)]

In [37]:
region_mappings = {354:'Oceania',
                   353:'Asia',
                   352:'Africa',
                   355:'Europe',
                   356:'North America',
                   357:'South America'}

### Get latitude and longitude coordinates

In [38]:
import geopandas as gpd

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Get the ISO codes that are present in the world shapefile
valid_iso_codes = world['iso_a3'].unique()

# Remove the rows with ISO codes that are not present in the world shapefile
top_countries_2021 = top_countries_2021[top_countries_2021['Source'].isin(valid_iso_codes)]

# Define a function to get the latitude and longitude of the centroid of a country
def get_centroid(iso_code):
    country_geometry = world.loc[world['iso_a3'] == iso_code, 'geometry'].values[0]
    centroid = country_geometry.centroid
    return pd.Series({'latitude': centroid.y, 'longitude': centroid.x})

# Apply the function to each ISO code in the DataFrame
top_countries_2021[['latitude', 'longitude']] = top_countries_2021['Source'].apply(get_centroid)

/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_80497/3164023484.py:3: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [39]:
top_countries_2021 = top_countries_2021[top_countries_2021['Target'].isin(valid_iso_codes)]
top_countries_2021[['latitude_target', 'longitude_target']] = top_countries_2021['Target'].apply(get_centroid)

In [40]:
top_countries_2021.drop(columns = ['latitude','longitude','latitude_target','longitude_target'], inplace = True)

In [42]:
top_countries_2021

,Source,Target,timeset,weight
899620,AFG,PAK,"<[2021, 2022]>",0.325328
899598,AFG,IND,"<[2021, 2022]>",0.285691
899651,AGO,CHN,"<[2021, 2022]>",0.612315
899678,AGO,IND,"<[2021, 2022]>",0.073086
899785,ALB,ITA,"<[2021, 2022]>",0.426262
...,...,...,...,...
921743,ZAF,USA,"<[2021, 2022]>",0.107442
921767,ZMB,CHN,"<[2021, 2022]>",0.392402
921765,ZMB,CHE,"<[2021, 2022]>",0.105572
921844,ZWE,ARE,"<[2021, 2022]>",0.433884


In [83]:
targets_df = top_countries_2021[['Target', 'latitude_target', 'longitude_target']].drop_duplicates()

In [86]:
targets_df.rename(columns = {'Target':'Source', 'latitude_target':'latitude', 'longitude_target':'longitude'}, inplace = True)

In [87]:
node_info = pd.concat([unique_countries_df, targets_df], ignore_index=True)

In [89]:
node_info.drop_duplicates(inplace = True)

In [90]:
node_info

,Source,latitude,longitude
0,AFG,33.856399,66.086690
1,AGO,-12.245869,17.470573
2,ALB,41.141353,20.032426
3,ARE,23.868634,54.206715
4,ARG,-35.446821,-65.175361
...,...,...,...
168,VUT,-15.542677,167.073751
169,YEM,15.913232,47.535045
170,ZAF,-28.947033,25.048014
171,ZMB,-13.395068,27.727592


In [96]:
country_counts = top_countries_2021['Target'].value_counts()
df_in_flows = pd.merge(node_info, country_counts, left_on ='Source', right_on='Target',how='left')
df_in_flows.fillna(0, inplace=True)

In [97]:
df_in_flows

,Source,latitude,longitude,count
0,AFG,33.856399,66.086690,0.0
1,AGO,-12.245869,17.470573,0.0
2,ALB,41.141353,20.032426,0.0
3,ARE,23.868634,54.206715,21.0
4,ARG,-35.446821,-65.175361,1.0
...,...,...,...,...
168,VUT,-15.542677,167.073751,0.0
169,YEM,15.913232,47.535045,0.0
170,ZAF,-28.947033,25.048014,3.0
171,ZMB,-13.395068,27.727592,1.0


In [101]:
df_with_regions = df_in_flows.merge(locations, left_on = "Source", right_on = "location_code")

In [105]:
df_with_regions

,Source,latitude,longitude,count,location_code,parent_id
0,AFG,33.856399,66.086690,0.0,AFG,353.0
1,AGO,-12.245869,17.470573,0.0,AGO,352.0
2,ALB,41.141353,20.032426,0.0,ALB,355.0
3,ARE,23.868634,54.206715,21.0,ARE,353.0
4,ARG,-35.446821,-65.175361,1.0,ARG,357.0
...,...,...,...,...,...,...
168,VUT,-15.542677,167.073751,0.0,VUT,354.0
169,YEM,15.913232,47.535045,0.0,YEM,353.0
170,ZAF,-28.947033,25.048014,3.0,ZAF,352.0
171,ZMB,-13.395068,27.727592,1.0,ZMB,352.0


In [106]:
df_with_regions['region'] = df_with_regions['parent_id'].map(region_mappings)

In [107]:
df_with_regions

,Source,latitude,longitude,count,location_code,parent_id,region
0,AFG,33.856399,66.086690,0.0,AFG,353.0,Asia
1,AGO,-12.245869,17.470573,0.0,AGO,352.0,Africa
2,ALB,41.141353,20.032426,0.0,ALB,355.0,Europe
3,ARE,23.868634,54.206715,21.0,ARE,353.0,Asia
4,ARG,-35.446821,-65.175361,1.0,ARG,357.0,South America
...,...,...,...,...,...,...,...
168,VUT,-15.542677,167.073751,0.0,VUT,354.0,Oceania
169,YEM,15.913232,47.535045,0.0,YEM,353.0,Asia
170,ZAF,-28.947033,25.048014,3.0,ZAF,352.0,Africa
171,ZMB,-13.395068,27.727592,1.0,ZMB,352.0,Africa


In [108]:
df_with_regions.to_csv('country_attributes.csv', index = False)